In [1]:
import json
import pandas as pd

def create_data_csv(p_dt, p_o, test=False):
    with open(p_dt)  as d:
        data = json.load(d)
    print(data.keys())
    
    data_img = pd.DataFrame(data['images'])
    print(data_img.shape)

    data_dict_img = dict(zip(data_img['id'], data_img['filename']))

    data_ano = pd.DataFrame(data['annotations'])
    print(data_ano.shape)
    data_ano['image'] = data_ano['image_id'].map(lambda x: data_dict_img[x])
    
    answer_space = list(data_ano['answer'].unique())
#     answer_space = list(data_ano['answer'].value_counts()[data_ano['answer'].value_counts()>1].keys())
    print(answer_space.index(answer_space[0]), answer_space[0])
    # data_ano['label'] = data_ano['answer'].map(lambda x: answer_space.index(x))
    if test:
        data_ano['answer'] = data_ano['answer'].map(lambda x: '1')
    else:
        data_ano['answer'] = data_ano['answer'].map(lambda x: [x])
        data_ano['dataset'] = data_ano['answer'].map(lambda x: 'vqa')
        data_ano['question_id'] = data_ano['id']
    data_ano.drop(['id', 'image_id'], axis=1).to_json(p_o, orient="records")
    
    return answer_space

In [2]:
p_dt = '/home/annt/vqa/BLIP/dataset/evjqa/train-data/evjvqa_train.json'
p_o = './annotation/train.json'
answer_space = create_data_csv(p_dt, p_o)

dict_keys(['images', 'annotations'])
(3763, 2)
(23785, 4)
0 the girl wearing glasses wears a red shirt


In [3]:
from data import create_dataset, create_sampler, create_loader
from data.vqa_dataset import vqa_collate_fn
import yaml

config = yaml.load(open('./configs/vqa.yaml', 'r'), Loader=yaml.Loader)

datasets = create_dataset('vqa', config) 
samplers = [None, None]

train_loader, test_loader = create_loader(datasets,samplers,
                                              batch_size=[config['batch_size_train'],config['batch_size_test']],
                                              num_workers=[4,4],is_trains=[True, False], 
                                              collate_fns=[vqa_collate_fn,None]) 

/home/annt/miniconda3/envs/vqa/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
for k in train_loader:
    print(k)
    break

[tensor([[[[-1.6609, -1.6901, -1.6901,  ..., -1.5295, -1.4127, -1.4565],
          [-1.3835, -1.6901, -1.7047,  ..., -1.3397, -0.6974, -1.1499],
          [-1.1207, -1.6755, -1.6901,  ..., -1.0769, -0.3324, -0.5806],
          ...,
          [ 0.0909,  0.0909,  0.0909,  ...,  0.7625,  0.6603,  0.5581],
          [ 0.0909,  0.0909,  0.0909,  ...,  0.4121,  0.3245,  0.1931],
          [ 0.0909,  0.0909,  0.0909,  ..., -0.0842, -0.2302, -0.3178]],

         [[-1.6020, -1.6320, -1.6320,  ..., -1.4820, -1.4369, -1.4669],
          [-1.3169, -1.6320, -1.6470,  ..., -1.2568, -0.6415, -1.1218],
          [-1.0467, -1.6170, -1.6320,  ..., -0.9417, -0.2063, -0.4764],
          ...,
          [ 0.1839,  0.1839,  0.1839,  ...,  0.2589,  0.1689,  0.0638],
          [ 0.1839,  0.1839,  0.1839,  ..., -0.0712, -0.1763, -0.2963],
          [ 0.1839,  0.1839,  0.1839,  ..., -0.4764, -0.5365, -0.6115]],

         [[-1.2811, -1.3096, -1.3096,  ..., -1.1532, -1.0394, -1.0678],
          [-1.0110, -1.3096, 

In [21]:
len(k)

5

In [23]:
k[0].shape

torch.Size([16, 3, 480, 480])

In [31]:
len(k[2]), len(k[3]), sum(k[4])

(41, 41, 41)

In [25]:
len(k[1]), k[1]

(16,
 ['what color is the pillow on the right?',
  'what is the female holding in her right hand?',
  'does there appear to be more spectators or more participants?',
  'the tiles are in what shape?',
  'what seafood is in the bowl?',
  'what fabric do the shoes in the picture appear to be made of?',
  'what is the number of the train?',
  'what is the animal seen on the shirt?',
  'how many forks?',
  'is the man alone?',
  'is he jumping?',
  'where are they eating?',
  'is there traffic on this road?',
  'is that a live human child leaning on the rear bicycle tire?',
  "what's the number on the batter's back?",
  'what color is the ball?'])

In [10]:
ls ./dataset/

val2014/


In [9]:
ls

annotation/         eda.ipynb                requirements.txt
BLIP.gif            eval_nocaps.py           SECURITY.md
CODE_OF_CONDUCT.md  eval_retrieval_video.py  train_caption.py
CODEOWNERS          LICENSE.txt              train_nlvr.py
cog.yaml            models/                  train_retrieval.py
configs/            predict.py               train_vqa.py
data/               pretrain.py              transform/
dataset/            __pycache__/             utils.py
demo.ipynb          README.md


In [32]:
ls

annotation/         eda.ipynb                requirements.txt
BLIP.gif            eval_nocaps.py           SECURITY.md
CODE_OF_CONDUCT.md  eval_retrieval_video.py  train_caption.py
CODEOWNERS          LICENSE.txt              train_nlvr.py
cog.yaml            models/                  train_retrieval.py
configs/            predict.py               train_vqa.py
data/               pretrain.py              transform/
dataset/            __pycache__/             utils.py
demo.ipynb          README.md


In [33]:
cd dataset/

/home/annt/vqa/BLIP/dataset


In [ ]:
# !pip install opendatasets

In [36]:
import opendatasets as od

url_vqa = 'https://www.kaggle.com/datasets/haschool/evjqa'
od.download(url_vqa)


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading evjqa.zip to ./evjqa


100%|██████████| 611M/611M [00:11<00:00, 56.1MB/s] 


In [37]:
ls

evjqa/  val2014/


In [39]:
ls  evjqa/train-data/

evjvqa_train.json  train-images/


In [ ]:
# !pip install datasets

In [3]:
from transformers import ViTFeatureExtractor, ViTModel
import torch
from datasets import load_dataset

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

inputs = feature_extractor(image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

No config specified, defaulting to: cats-image/image


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 410.48it/s]


Dataset cats-image downloaded and prepared to /home/annt/.cache/huggingface/datasets/huggingface___cats-image/image/1.9.0/68fbc793fb10cd165e490867f5d61fa366086ea40c73e549a020103dcb4f597e. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 294.36it/s]


[1, 197, 768]

In [10]:
outputs.pooler_output.size()[:-1]

torch.Size([1])

In [12]:
inputs.keys()

dict_keys(['pixel_values'])

In [16]:
from datasets import load_dataset, set_caching_enabled
from transformers import (
    # Preprocessing / Common
    AutoTokenizer, AutoFeatureExtractor,
    # Text & Image Models (Now, image transformers like ViTModel, DeiTModel, BEiT can also be loaded using AutoModel)
    AutoModel,            
    # Training / Evaluation
    TrainingArguments, Trainer,
    # Misc
    logging
)
# SET CACHE FOR HUGGINGFACE TRANSFORMERS + DATASETS
os.environ['HF_HOME'] = os.path.join(".", "cache")
# SET ONLY 1 GPU DEVICE
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

set_caching_enabled(True)
logging.set_verbosity_error()

In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
#     print('Memory Usage:')
#     print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
#     print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

cuda:0
NVIDIA A40
